In [30]:
# Import standard libraries
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from importlib import reload
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
from sklearn.metrics import adjusted_mutual_info_score
import seaborn as sns
from scipy import sparse
import scipy.stats as ss
import csv
import pandas as pd
import networkx as nx
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import requests

In [2]:
# Specify all directories
# Directory of adhesome data
dir_adhesome = '/home/louiscam/projects/gpcr/data/adhesome_data/'
# Directory of genome data
dir_genome = '/home/louiscam/projects/gpcr/data/genome_data/'
# Directory of TF target data
dir_htftarget = '/home/louiscam/projects/gpcr/data/tf_data/hTFtarget/'
# Saving directory
saving_dir = '/home/louiscam/projects/gpcr/save/figures/'

# Load hTFtarget metadata

In [14]:
# Load and clean metadata
tf_metadata_df = pd.read_csv(dir_htftarget+'hTFtarget_datadescription.csv', header=0)
tf_metadata_df['Dataset'] = tf_metadata_df['Dataset'].str.replace('Dataset-','').astype(int)
tf_metadata_df['Tissue'] = tf_metadata_df['Tissue'].str.lower()

In [17]:
# Restrict to tissues of interest
tissues_of_interest = ['breast','lung','blood','bone marrow','liver', 'kidney', 'colon', 'brain', 'skin']
tf_metadata_df = tf_metadata_df[tf_metadata_df['Tissue'].isin(tissues_of_interest)]

In [19]:
tf_metadata_df.head()

,Dataset,TF,Cell line,Tissue,Disease
131,306,BRD7,BJ,skin,Normal
132,307,BRD7,BJ,skin,Normal
133,520,CTCF,WERI-Rb-1,skin,Retinoblastoma
134,521,CTCF,BJ,skin,Normal
135,549,CTCF,AG04449,skin,Normal


# Scrape more metadata from hTFtarget

Goal: for each data set ID in tf_metadata_df, fetch from the hTFtarget website (http://bioinfo.life.hust.edu.cn/hTFtarget/#!/tf) the following information
- ID
- TF
- Project
- Cell Line
- Cell Type
- Tissue

In [29]:
# configure webdriver to use Chrome browser
# see https://stackoverflow.com/questions/29858752/
# error-message-chromedriver-executable-needs-to-be-available-in-the-path
driver = webdriver.Chrome('/path/to/chromedriver')

In [31]:
URL = "http://bioinfo.life.hust.edu.cn/hTFtarget/#!/tf"
page = requests.get(URL)

In [32]:
print(page.text)

<!doctype html>
<html lang="en" ng-app="hTFtarget">
<head>

    <title>hTFtarget</title>

    <link rel="icon" href="/static/hTFtarget/img/hTFtarget-icon.ico">


    <link rel="stylesheet" href="/static/hTFtarget/css/bootstrap.css">
    <link rel="stylesheet" href="/static/hTFtarget/lib/bootstrap-slider/css/bootstrap-slider.css">
    <link rel="stylesheet" href="/static/hTFtarget/lib/angular-intro/introjs.min.css">
    <link rel="stylesheet" href="/static/hTFtarget/lib/angular-tablesort/tablesort.css">
    <link rel="stylesheet" href="/static/hTFtarget/lib/angular-treeview/css/angular.treeview.css">
    <link rel="stylesheet" href="/static/hTFtarget/lib/bootstrap-grid-flexbox/css/bootstrap-grid-flexbox.css">
    <link rel="stylesheet" href="/static/hTFtarget/css/main.css">
    <link rel="stylesheet" href="/static/hTFtarget/css/navbar-customize.css">


    <script src="/static/hTFtarget/lib/angular/angular.js"></script>
    <script src="/static/hTFtarget/lib/angular/angular-route.js"></